# 文章から情報を抽出！Rasaでエンティティ抽出入門

チャットボット開発フレームワークのRasaを用いて、エンティティ抽出をする

## 環境構築

In [1]:
# Rasa と GiNZA をインストール
# Rasaの内部でspacyを使用する際の言語モデルも一緒にインストール （ ja-ginza : GiNZAの日本語モデル )
! pip install rasa==3.2.8 ja-ginza==5.1.2

# NOTE 終了後、ランタイムの再起動が必要

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# 共通的なモジュールのインポート
import numpy as np
import pandas as pd

## エンティティ抽出の方法
- 2種類のファイルが必要
    - 設定ファイル（pipelineのconfig）
    - 訓練サンプルファイル（エンティティ抽出のアノテーション付与）
- 手順
    1. `rasa.model_training.train_nlu()`でモデルを訓練
    2. `rasa.core.agent.Agent`クラスを生成
    3. `Agent.load_model()`により訓練済みモデルを読込
    4. `Agent.parse_message()`で文章を解析

    

In [3]:
# 設定ファイルと、訓練サンプルファイルを生成する

# 設定ファイル
# NOTE SpacyNLPのmodelを指定する必要あり
#       （指定しない場合、エラー"InvalidModelError: Missing model configuration for `SpacyNLP` in `config.yml`."が発生する）
model_config = \
"""recipe: default.v1
language: ja

pipeline:
  - name: SpacyNLP
    model: ja_ginza
  - name: SpacyTokenizer
  - name: SpacyFeaturizer
  - name: RegexFeaturizer
  - name: CRFEntityExtractor
"""

# 訓練サンプルファイル
# cf. 
# - https://rasa.com/docs/rasa/training-data-format/#entities
# - https://rasa.com/docs/rasa/nlu-training-data/#entities-roles-and-groups
train_data = \
"""nlu:
- intent: eat_out
  examples: |
    - [昨日]{"entity": "date"}は[両親]{"entity": "companion"}と一緒に[銀座]{"entity": "place"}の[寿司]{"entity": "food_genre"}を食べにいきました。
    - [おととい]{"entity": "date"}、[友人]{"entity": "companion"}と一緒に[中華料理]{"entity": "food_genre"}を食べに、[横浜]{"entity": "place"}に行った。
    - [一週間前]{"entity": "date"}に、[祖父母]{"entity": "companion"}と[日本食]{"entity": "food_genre"}を食べました。
    - [今日]{"entity": "date"}は[弟]{"entity": "companion"}に[台湾料理]{"entity": "food_genre"}に連れて行ってもらう。
    - [明日]{"entity": "date"}、[恋人]{"entity": "companion"}と[恵比寿]{"entity": "place"}に[イタリアン]{"entity": "food_genre"}を食べに行く予定。
"""

open('./config_ja.yml', 'w').write(model_config)
open('./train_data_ja.yml', 'w').write(train_data)

605

In [4]:
# モデルを訓練
# https://rasa.com/docs/rasa/reference/rasa/model_training
from rasa.model_training import train_nlu

train_nlu(
    config='config_ja.yml',
    nlu_data='train_data_ja.yml',
    output='sample_model_ja')

/usr/local/lib/python3.7/dist-packages/tornado/httputil.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  class HTTPHeaders(collections.MutableMapping):
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


Your Rasa model is trained and saved at 'sample_model_ja/nlu-20220915-141527-inclusive-sofa.tar.gz'.


'sample_model_ja/nlu-20220915-141527-inclusive-sofa.tar.gz'

In [5]:
# モデルを読み込んで、文章を解析
from rasa.core.agent import Agent
from rasa.model import get_local_model

# Agentを生成
agent = Agent()

# 訓練モデルを読み込み
model_path = get_local_model('sample_model_ja')
agent.load_model(model_path)

# 文章を解析（エンティティ抽出）
await agent.parse_message('昨日、友達と、恵比寿のイタリアンに行った。')

{'text': '昨日、友達と、恵比寿のイタリアンに行った。',
 'intent': {'name': None, 'confidence': 0.0},
 'entities': [{'entity': 'date',
   'start': 0,
   'end': 2,
   'confidence_entity': 0.8748535803053381,
   'value': '昨日',
   'extractor': 'CRFEntityExtractor'},
  {'entity': 'companion',
   'start': 3,
   'end': 5,
   'confidence_entity': 0.7098377521624063,
   'value': '友達',
   'extractor': 'CRFEntityExtractor'},
  {'entity': 'place',
   'start': 7,
   'end': 10,
   'confidence_entity': 0.739316057201231,
   'value': '恵比寿',
   'extractor': 'CRFEntityExtractor'},
  {'entity': 'food_genre',
   'start': 11,
   'end': 16,
   'confidence_entity': 0.5424315319442129,
   'value': 'イタリアン',
   'extractor': 'CRFEntityExtractor'}],
 'text_tokens': [(0, 2),
  (2, 3),
  (3, 5),
  (5, 6),
  (6, 7),
  (7, 10),
  (10, 11),
  (11, 16),
  (16, 17),
  (17, 19),
  (19, 20),
  (20, 21)]}

In [6]:
# 訓練サンプルが少ないので、訓練サンプルに含まれない場所やジャンルは認識が難しい様子
await agent.parse_message('明後日、家族と、東京のフレンチに行く予定です。')

{'text': '明後日、家族と、東京のフレンチに行く予定です。',
 'intent': {'name': None, 'confidence': 0.0},
 'entities': [{'entity': 'date',
   'start': 0,
   'end': 3,
   'confidence_entity': 0.6910108163613534,
   'value': '明後日',
   'extractor': 'CRFEntityExtractor'},
  {'entity': 'companion',
   'start': 4,
   'end': 6,
   'confidence_entity': 0.7092017421917339,
   'value': '家族',
   'extractor': 'CRFEntityExtractor'}],
 'text_tokens': [(0, 3),
  (3, 4),
  (4, 6),
  (6, 7),
  (7, 8),
  (8, 10),
  (10, 11),
  (11, 15),
  (15, 16),
  (16, 18),
  (18, 20),
  (20, 22),
  (22, 23)]}

## 訓練サンプルと精度の関係性

- 知りたいこと：どのくらい訓練データが必要なのか？  
- パラメータ  
    1. 訓練サンプル数
    2. 文章の型の種類
    3. 各エンティティの種類

### 訓練・評価サンプルの準備

In [7]:
# 飲食店ジャンル： https://www.synchro-food.co.jp/news/press/1017

# 文章の型 
train_templates = [
    "(date)は、(companion)と一緒に(place)の(food_genre)を食べにいきます。",
    "(date)行った(food_genre)で食べた(dish_name)は本当に美味しかった。",
    "(date)は(companion)に(food_genre)に連れて行ってもらう。",
    "(date)、(companion)と(place)に(food_genre)を食べに行く予定です。",
    "(food_genre)で、(dish_name)を食べました。",
    "(place)の(food_genre)はおいしい。",
    "(dish_name)を食べに、(food_genre)に行きました。",
    "(place)にある(food_genre)で、(dish_name)を食べた。",
    "(date)、(companion)と(food_genre)を食べました。",
    "(date)、(companion)と一緒に(food_genre)を食べに、(place)に行った。",
]
val_templates = [
    "(date)は(food_genre)を食べに行った。",
    "(companion)と(food_genre)で(dish_name)をたらふく食べた。",
    "(date)は、(place)の駅前にある(food_genre)に行った。",
    "(dish_name)の有名な(food_genre)に、(companion)と(date)行きました。",
    "(place)の(food_genre)は(dish_name)がオススメらしいので、(date)(companion)と行きます。",
    "(place)で、(dish_name)を食べました。",             # 訓練サンプルと類似。food_genre と place が違うだけ。難しい？
    "(date)、(companion)と(dish_name)を食べました。",  # 訓練サンプルと類似。food_genre と dish_name が違うだけ。難しい？
]

# エンティティの要素
train_entity_values = {
    'date' : ['今日', '昨日', '一昨日', '3日前', '一週間前', '先週', '明日', '明後日', '来週'],
    'companion' : ['友人', '家族', '両親', '父親', '母親', '父', '母', '祖父母', '祖父', '祖母', '恋人', '彼氏', '彼女', '上司', '部下', '同僚'],
    'food_genre' : ['イタリアン', 'フレンチ', '中華', '中華料理', '和食', '日本食', '寿司', '懐石料理', '焼肉', '居酒屋', 'スペインバル', '韓国料理', 'エスニック料理'],
    'dish_name' : ['パスタ', 'ピザ', '酢豚', '小籠包', '麻婆豆腐', '天ぷら', 'そば', 'うどん', 'とんかつ', '刺身', '生姜焼き', '野菜炒め', '焼き魚', 'ビビンバ', '冷麺', 'カルパッチョ', '純豆腐', 'チゲ鍋'],
    'place' : ['池袋', '大塚', '巣鴨', '駒込', '田端', '西日暮里', '日暮里', '鶯谷', '上野', '御徒町', '秋葉原', '神田', '東京', '有楽町', '新橋', '浜松町', '田町', '高輪ゲートウェイ', '品川', '大崎', '五反田', '目黒', '恵比寿', '渋谷', '原宿', '代々木', '新宿', '新大久保', '高田馬場', '目白'],
}
val_entity_values = {
    'date' : ['本日', 'きのう', '2日前', '先日', '最近', '一ヶ月前', '今度', 'あさって', 'いつか', '将来'],
    'companion' : ['友だち', '知人', '父さん', '母さん', '兄', '姉', '弟', '妹', '先輩', '後輩', 'お客さん'],
    'food_genre' : ['カフェ', '喫茶店', '焼き鳥', 'インド料理', 'タイ料理', '鉄板焼き', '洋食', '立ち飲み屋', '創作料理', 'ラーメン屋'],
    'dish_name' : ['ミルクレープ', 'フレンチトースト', 'つくね', 'タンドリーチキン', 'ビーフン', 'フォー', 'お好み焼き', 'とんこつラーメン'],
    'place' : ['札幌', '仙台', 'さいたま', '横浜', '名古屋', '京都', '大阪', '神戸', '広島', '博多'],
}

def generate_train_samples(templates, entity_values, num_samples, seed=123):
    np.random.seed(seed)
    samples = []
    while len(samples) < num_samples:
        template = templates[len(samples) % len(templates)]
        for name, values in entity_values.items():
            if f'({name})' in template:
                value = np.random.choice(values)
                template = template.replace(f'({name})', f'[{value}]{{"entity": "{name}"}}')
        samples.append(template)
    return samples

def generate_val_samples(templates, entity_values, num_samples, seed=123):
    np.random.seed(seed)
    samples = []
    while len(samples) < num_samples:
        template = templates[len(samples) % len(templates)]
        ann = {}  # 正解エンティティを格納
        for name, values in entity_values.items():
            if f'({name})' in template:
                value = np.random.choice(values)
                template = template.replace(f'({name})', value)  # 単純に単語で置き換え
                ann[name] = value
        samples.append([template, ann])  # 文章と正解エンティティのセット
    return samples

train_samples_demo = generate_train_samples(train_templates, train_entity_values, num_samples=5, seed=1)
val_samples_demo = generate_val_samples(val_templates, val_entity_values, num_samples=5)

print(train_samples_demo)
print(val_samples_demo)

['[先週]{"entity": "date"}は、[彼氏]{"entity": "companion"}と一緒に[上野]{"entity": "place"}の[エスニック料理]{"entity": "food_genre"}を食べにいきます。', '[先週]{"entity": "date"}行った[イタリアン]{"entity": "food_genre"}で食べた[純豆腐]{"entity": "dish_name"}は本当に美味しかった。', '[昨日]{"entity": "date"}は[彼女]{"entity": "companion"}に[懐石料理]{"entity": "food_genre"}に連れて行ってもらう。', '[明日]{"entity": "date"}、[祖母]{"entity": "companion"}と[五反田]{"entity": "place"}に[中華]{"entity": "food_genre"}を食べに行く予定です。', '[日本食]{"entity": "food_genre"}で、[野菜炒め]{"entity": "dish_name"}を食べました。']
[['2日前は焼き鳥を食べに行った。', {'date': '2日前', 'food_genre': '焼き鳥'}], ['弟と喫茶店でタンドリーチキンをたらふく食べた。', {'companion': '弟', 'food_genre': '喫茶店', 'dish_name': 'タンドリーチキン'}], ['将来は、仙台の駅前にある洋食に行った。', {'date': '将来', 'food_genre': '洋食', 'place': '仙台'}], ['ミルクレープの有名なラーメン屋に、知人と本日行きました。', {'date': '本日', 'companion': '知人', 'food_genre': 'ラーメン屋', 'dish_name': 'ミルクレープ'}], ['名古屋のインド料理はお好み焼きがオススメらしいので、本日後輩と行きます。', {'date': '本日', 'companion': '後輩', 'food_genre': 'インド料理', 'dish_name': 'お好み焼き', 'place': '名古屋'}]]


In [8]:
# 訓練サンプルをファイルに書き出す
TEXT_BASE = \
"""nlu:
- intent: eat_out
  examples: |
"""
def output_train_samples(yml_filepath, train_samples):
    assert yml_filepath.endswith('.yml')
    text_base = TEXT_BASE
    text_base += '\n'.join([f'    - {text}' for text in train_samples])
    open(yml_filepath, 'w').write(text_base)
    print('Saved ...', yml_filepath)

output_train_samples('train_data_ja_demo.yml', train_samples_demo)

Saved ... train_data_ja_demo.yml


### 評価用スクリプト

In [9]:
async def extract_entities(agent, message):
    """エンティティ抽出."""
    # 推論
    pred_all = await agent.parse_message(message)
    # エンティティの推定結果のみを抽出
    pred_entities = {}
    for pred in pred_all['entities']:
        entity = pred['entity']
        value = pred['value']
        score = pred['confidence_entity']
        # 同一のエンティティが複数抽出された場合は、スコアが高いものを採用.
        if entity in pred_entities.keys():
            _, s = pred_entities[entity]
            if s >= score:
                continue
        pred_entities[entity] = (value, score)
    return pred_entities

# 動作テスト
await extract_entities(agent, '昨日、友達と、恵比寿のイタリアンに行った。')

{'date': ('昨日', 0.8748535803053381),
 'companion': ('友達', 0.7098377521624063),
 'place': ('恵比寿', 0.739316057201231),
 'food_genre': ('イタリアン', 0.5424315319442129)}

In [10]:
async def eval(agent, val_samples):
    """エンティティ抽出の正解率を評価する."""
    entity_names = ['date', 'companion', 'food_genre', 'dish_name', 'place']
    eval_summary = {e : [0, 0] for e in entity_names}  # (gt_num, correct_num)
    for message, gt_entities in val_samples:
        pred_entities = await extract_entities(agent, message)
        for entity, gt_value in gt_entities.items():
            # gt_num を加算
            eval_summary[entity][0] += 1
            # 推定結果と照合
            if (entity in pred_entities.keys()):
                pred_value, score = pred_entities[entity]
                if pred_value == gt_value:
                    # correct_num を加算
                    eval_summary[entity][1] += 1
    # エンティティ毎に正解率を算出
    acc_each = {entity : correct_num / gt_num for (entity, (gt_num, correct_num)) in eval_summary.items()}
    # 全体の正解率
    acc_total = np.sum([n2 for _, n2 in eval_summary.values()]) / np.sum([n1 for n1, _ in eval_summary.values()])
    return acc_total, acc_each

# 動作テスト
val_samples_200 = generate_val_samples(val_templates, val_entity_values, num_samples=200)
await eval(agent, val_samples_200)

(0.25,
 {'date': 0.46153846153846156,
  'companion': 0.5263157894736842,
  'food_genre': 0.13194444444444445,
  'dish_name': 0.0,
  'place': 0.1411764705882353})

### 定性評価

In [11]:
# 訓練サンプルの作成
train_samples_exp = generate_train_samples(train_templates, train_entity_values, num_samples=300, seed=1)
output_train_samples('train_data_ja_exp.yml', train_samples_exp)
# 訓練
train_nlu(
    config='config_ja.yml',
    nlu_data='train_data_ja_exp.yml',
    output='sample_model_ja_exp')
# モデル読み込み
agent = Agent()
model_path = get_local_model('sample_model_ja_exp')
agent.load_model(model_path)

Saved ... train_data_ja_exp.yml
Your Rasa model is trained and saved at 'sample_model_ja_exp/nlu-20220915-141540-optimal-ricotta.tar.gz'.


In [12]:
# 訓練サンプルにない、文章の型、と、エンティティ値
message = "横浜のラーメン屋はとんこつラーメンがオススメらしいので、あさって先輩と行きます。"
await extract_entities(agent, message)

{'place': ('横浜', 0.9606404673208094), 'companion': ('先輩', 0.8571427007748562)}

In [13]:
# 訓練サンプルにない、文章の型、と、エンティティ値
message = "知人とラーメン屋でとんこつラーメンをたらふく食べた。"
await extract_entities(agent, message)

{'companion': ('知人', 0.7614032783964635),
 'dish_name': ('ラーメン', 0.6512579042011245)}

In [14]:
# 訓練サンプルにない、文章の型、と、エンティティ値
message = "先日、仙台の駅前にあるインド料理に行った。"
await extract_entities(agent, message)

{'date': ('先日', 0.6532071851633521),
 'place': ('仙台', 0.933608850518393),
 'food_genre': ('インド料理', 0.9879440559052248)}

In [15]:
# 訓練サンプルにない、文章の型、と、エンティティ値
message = "フレンチトーストの有名な喫茶店に、姉と2日前に行きました。"
await extract_entities(agent, message)

{'place': ('フレンチトースト', 0.9586308410736651),
 'companion': ('姉', 0.9568551934457641),
 'date': ('2日前', 0.6744999438127921)}

In [16]:
# 訓練サンプルにエンティティタイプが異なる文章①
# - 訓練 : (food_genre)で、(dish_name)を食べました。
# - 評価 : (place)で、(dish_name)を食べました。
message = "広島で、お好み焼きを食べました。"
await extract_entities(agent, message)

{'date': ('広島', 0.29006321747298736),
 'dish_name': ('お好み焼き', 0.760606985896815)}

In [17]:
# 訓練サンプルにエンティティタイプが異なる文章②
# - 訓練 : (date)、(companion)と(food_genre)を食べました。
# - 評価 : (date)、(companion)と(dish_name)を食べました。
message = "きのう、友だちとフレンチトーストを食べました。"
await extract_entities(agent, message)

{'date': ('きのう', 0.6534438061631824),
 'companion': ('友だち', 0.9569673718474322),
 'food_genre': ('フレンチトースト', 0.36414176521743385)}

### 評価１. 訓練サンプル数と精度


In [18]:
eval_all1 = []
for train_num in (10, 50, 100, 300, 500, 1000):
    # 訓練データ生成
    train_samples_exp = generate_train_samples(train_templates, train_entity_values, num_samples=train_num, seed=1)
    output_train_samples('train_data_ja_exp.yml', train_samples_exp)
    # 訓練
    train_nlu(
        config='config_ja.yml',
        nlu_data='train_data_ja_exp.yml',
        output='sample_model_ja_exp')
    # 評価
    agent = Agent()
    model_path = get_local_model('sample_model_ja_exp')
    agent.load_model(model_path)
    acc_total, acc_each = await eval(agent, val_samples_200)
    # 格納
    eval_res = acc_each.copy()
    eval_res['sample_num'] = train_num
    eval_res['total'] = acc_total
    eval_all1.append(eval_res)

df_eval1 = pd.DataFrame(eval_all1)
df_eval1[['sample_num', 'total', 'date', 'companion', 'food_genre', 'dish_name', 'place']]

Saved ... train_data_ja_exp.yml


/usr/local/lib/python3.7/dist-packages/rasa/shared/nlu/training_data/features.py:152: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  f_as_text = self.features.tostring()


Your Rasa model is trained and saved at 'sample_model_ja_exp/nlu-20220915-141547-nippy-sled.tar.gz'.
Saved ... train_data_ja_exp.yml


/usr/local/lib/python3.7/dist-packages/rasa/shared/nlu/training_data/features.py:152: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  f_as_text = self.features.tostring()


Your Rasa model is trained and saved at 'sample_model_ja_exp/nlu-20220915-141600-many-jug.tar.gz'.
Saved ... train_data_ja_exp.yml


...

/usr/local/lib/python3.7/dist-packages/rasa/shared/nlu/training_data/features.py:152: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  f_as_text = self.features.tostring()


Your Rasa model is trained and saved at 'sample_model_ja_exp/nlu-20220915-141652-deterministic-motion.tar.gz'.
Saved ... train_data_ja_exp.yml


/usr/local/lib/python3.7/dist-packages/rasa/shared/nlu/training_data/features.py:152: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  f_as_text = self.features.tostring()


Your Rasa model is trained and saved at 'sample_model_ja_exp/nlu-20220915-141725-espressivo-influencer.tar.gz'.


,sample_num,total,date,companion,food_genre,dish_name,place
0,10,0.302548,0.461538,0.464912,0.298611,0.000000,0.329412
1,50,0.535032,0.615385,0.964912,0.416667,0.147887,0.670588
2,100,0.597134,0.615385,0.964912,0.416667,0.422535,0.670588
3,300,0.587580,0.615385,0.964912,0.375000,0.422535,0.670588
4,500,0.570064,0.615385,0.964912,0.298611,0.422535,0.670588
5,1000,0.597134,0.615385,0.964912,0.298611,0.542254,0.670588


### 評価２. 文章の型の種類数と精度



In [19]:
eval_all2 = []
for template_num in (3, 5, 7, 10):
    exp_templates = train_templates[:template_num]
    # 訓練データ生成
    train_num = 300
    train_samples_exp = generate_train_samples(exp_templates, train_entity_values, num_samples=train_num, seed=1)
    output_train_samples('train_data_ja_exp.yml', train_samples_exp)
    # 訓練
    train_nlu(
        config='config_ja.yml',
        nlu_data='train_data_ja_exp.yml',
        output='sample_model_ja_exp')
    # 評価
    agent = Agent()
    model_path = get_local_model('sample_model_ja_exp')
    agent.load_model(model_path)
    acc_total, acc_each = await eval(agent, val_samples_200)
    # 格納
    eval_res = acc_each.copy()
    eval_res['template_num'] = template_num
    eval_res['total'] = acc_total
    eval_all2.append(eval_res)

df_eval2 = pd.DataFrame(eval_all2)
df_eval2[['template_num', 'total', 'date', 'companion', 'food_genre', 'dish_name', 'place']]

Saved ... train_data_ja_exp.yml
Your Rasa model is trained and saved at 'sample_model_ja_exp/nlu-20220915-141804-sophisticated-fleet.tar.gz'.
Saved ... train_data_ja_exp.yml
Your Rasa model is trained and saved at 'sample_model_ja_exp/nlu-20220915-141820-oriented-surfer.tar.gz'.
Saved ... train_data_ja_exp.yml
Your Rasa model is trained and saved at 'sample_model_ja_exp/nlu-20220915-141834-inclusive-buffer.tar.gz'.
Saved ... train_data_ja_exp.yml
Your Rasa model is trained and saved at 'sample_model_ja_exp/nlu-20220915-141849-merciless-remote.tar.gz'.


,template_num,total,date,companion,food_genre,dish_name,place
0,3,0.343949,0.608392,0.500000,0.298611,0.000000,0.341176
1,5,0.482484,0.601399,0.964912,0.298611,0.246479,0.341176
2,7,0.554140,0.615385,0.964912,0.298611,0.352113,0.670588
3,10,0.587580,0.615385,0.964912,0.375000,0.422535,0.670588


### 評価３. エンティティの種類数と精度

In [20]:
eval_all3 = []
for entity_num in (3, 5, 7, 9, -1):
    # エンティティの種類を制限
    if entity_num < 0:
        exp_entity_values = train_entity_values.copy()
    else:
        exp_entity_values = {name: values[:entity_num] for name, values in train_entity_values.items()}
    # 訓練データ生成
    train_num = 300
    train_samples_exp = generate_train_samples(train_templates, exp_entity_values, num_samples=train_num, seed=1)
    output_train_samples('train_data_ja_exp.yml', train_samples_exp)
    # 訓練
    train_nlu(
        config='config_ja.yml',
        nlu_data='train_data_ja_exp.yml',
        output='sample_model_ja_exp')
    # 評価
    agent = Agent()
    model_path = get_local_model('sample_model_ja_exp')
    agent.load_model(model_path)
    acc_total, acc_each = await eval(agent, val_samples_200)
    # 格納
    eval_res = acc_each.copy()
    eval_res['entity_num'] = entity_num if entity_num > 0 else 'all'
    eval_res['total'] = acc_total
    eval_all3.append(eval_res)

df_eval3 = pd.DataFrame(eval_all3)
df_eval3[['entity_num', 'total', 'date', 'companion', 'food_genre', 'dish_name', 'place']]

Saved ... train_data_ja_exp.yml
Your Rasa model is trained and saved at 'sample_model_ja_exp/nlu-20220915-141904-warped-resultant.tar.gz'.
Saved ... train_data_ja_exp.yml
Your Rasa model is trained and saved at 'sample_model_ja_exp/nlu-20220915-141919-scalding-vertex.tar.gz'.
Saved ... train_data_ja_exp.yml
Your Rasa model is trained and saved at 'sample_model_ja_exp/nlu-20220915-141934-celeste-ball.tar.gz'.
Saved ... train_data_ja_exp.yml
Your Rasa model is trained and saved at 'sample_model_ja_exp/nlu-20220915-141949-tan-rehab.tar.gz'.
Saved ... train_data_ja_exp.yml
Your Rasa model is trained and saved at 'sample_model_ja_exp/nlu-20220915-142005-cross-filler.tar.gz'.


,entity_num,total,date,companion,food_genre,dish_name,place
0,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,5,0.232484,0.153846,0.464912,0.298611,0.000000,0.329412
2,7,0.490446,0.615385,0.912281,0.298611,0.112676,0.670588
3,9,0.507962,0.615385,0.964912,0.298611,0.147887,0.670588
4,all,0.587580,0.615385,0.964912,0.375000,0.422535,0.670588


### 参考）訓練サンプルに含めたらちゃんと推定できる？

In [21]:
# 訓練データ生成
train_samples_exp = generate_train_samples(
    train_templates + val_templates,
    {entity: train_entity_values[entity] + val_entity_values[entity] for entity in train_entity_values.keys()},
    num_samples=500, seed=1)
output_train_samples('train_data_ja_exp.yml', train_samples_exp)
# 訓練
train_nlu(
    config='config_ja.yml',
    nlu_data='train_data_ja_exp.yml',
    output='sample_model_ja_exp')
# 評価
agent = Agent()
model_path = get_local_model('sample_model_ja_exp')
agent.load_model(model_path)
acc_total, acc_each = await eval(agent, val_samples_200)
print(acc_total)
print(acc_each)

Saved ... train_data_ja_exp.yml
Your Rasa model is trained and saved at 'sample_model_ja_exp/nlu-20220915-142022-quiet-click.tar.gz'.
0.9888535031847133
{'date': 0.972027972027972, 'companion': 0.9736842105263158, 'food_genre': 1.0, 'dish_name': 1.0, 'place': 1.0}


In [22]:
# 訓練サンプルにない例
message = "昨日ね、モンブランが有名な表参道のケーキ屋にお姉ちゃんと行ったんだけど、本当においしくてびっくりした。"
await extract_entities(agent, message)

{'date': ('昨日', 0.9518796825800208),
 'dish_name': ('モンブラン', 0.4914582120213147),
 'place': ('表参道', 0.4691676646007581),
 'food_genre': ('ケーキ屋', 0.9592409658184005),
 'companion': ('お姉ちゃん', 0.9347256843599344)}

In [25]:
# 訓練サンプルにない例
message = "お得意さんに連れられて両国の居酒屋で、串カツをいただいた。"
await extract_entities(agent, message)

{'date': ('お得意', 0.46694943534305905),
 'companion': ('さん', 0.41406724921855526),
 'place': ('両国', 0.416296922625642),
 'food_genre': ('居酒屋', 0.9965381398730863),
 'dish_name': ('串カツ', 0.5878260743967351)}